# Omission Detection
This script reads in a file of transcripts and genderated notes and identifies the omissions for each pair.
It then grades the omission on severity of omission.




In [1]:
import os
import re
import json
import base64
import requests
import time
import urllib.parse

import pandas as pd
import numpy as np
import io
import matplotlib.pyplot as plt

from IPython.display import Markdown
#os.environ['PATH'] = path_to_add + os.pathsep + os.environ['PATH']
#if path_to_add not in sys.path:
#  sys.path.append(path_to_add)
from openai import AzureOpenAI



# First, let's test by hard coding your Mulesoft Azure API key into the next line
API_KEY = 'MY_KEY='  ##### Paste your API key between the quotes #####
API_VERSION = '2024-06-01'  # For the most recent production release: https://learn.microsoft.com/en-us/azure/ai-services/openai/api-version-deprecation#latest-ga-api-release
RESOURCE_ENDPOINT = 'https://unified-api.ucsf.edu/general'  # no trailing slash--this is used by libraries as a partial URLUse 

In [8]:
file_path = r"YOUR_PATH"
df = pd.read_csv(file_path)

Set up Versa Client

In [9]:
client = AzureOpenAI(
    api_key=API_KEY,
    api_version= API_VERSION,
    azure_endpoint=RESOURCE_ENDPOINT,
    # azure_deployment=deployment,
)


client

In [29]:
# This section copies the sample data provided to a new dataframe for analysis
# For each encounter, we identify the list of omissions from the HPI
final_try_df = df
for index, row in final_try_df.iterrows():
    final_try_df.at[index, 'Omissions'] = IdentifyOmission7(row['Transcript'], row['HPI'])


In [202]:
## For each encounter, classify the severity admissions that were identified
## this also does a second filter for only including items that should be in the HPI
final_try_df['Classified'] = None
for index, row in final_try_df.iterrows():
    final_try_df.at[index, 'Classified'] = FilterAndFormatFinal(row['Omissions'])

In [203]:
## This section calculates the severity scoring for omissions of each encounter
## There are two scores, the Severity Score calculates the score based on number and severity of omissions
## The Weighted Severity Score takes into account the length of the transcript
final_try_df['Transcript_Char_Count'] = final_try_df['Transcript'].str.len()
final_try_df['Severity_Score'] = final_try_df['Classified'].apply(calculate_severity_score)
final_try_df['Weighted_Severity_Score'] = final_try_df['Severity_Score'] / (final_try_df['Transcript_Char_Count'] / 10000)


In [196]:
### This calculates the
def calculate_severity_score(csv_string: str):
    """
    Calculate a total severity score based on a CSV string containing omission data.

    This function parses a CSV-formatted string containing a list of clinical omissions,
    each with an associated severity level. It maps severity levels to numeric values:
        - 'Critical' = 3
        - 'Moderate' = 1
        - 'Optional' = 0

    The total score is the sum of all mapped severity values. If the input string is empty,
    the function returns 0. If there is a parsing error, it prints the error and returns -7.

    Parameters:
        csv_string (str): A string containing CSV-formatted data with a 'Severity' column.

    Returns:
        int: The total severity score, or -7 if a parsing error occurs.
    """
    if csv_string == "":
        print("Empty String")
        return 0
    try:
        total_score = 0
        temp_df = pd.read_csv(io.StringIO(csv_string), header=1)
        # Map severity levels to scores
        severity_scores = {
            'Critical': 3,
            'Moderate': 1,
            'Optional': 0
        }

        temp_df['Score'] = temp_df['Severity'].map(severity_scores)
        total_score = temp_df['Score'].sum()
        return total_score
    
    except Exception as e:
        # Handle any parsing errors
        print(f"Error processing CSV data: {e}")
        print(csv_string)
        return -7



In [184]:

def FilterAndFormatFinal(omis: str):
    """
    Sends a list of clinical omissions to a language model to classify each as 
    either 'Critical', 'Moderate', or 'Optional' based on its clinical significance. It 
    returns a properly formatted, comma-delimited CSV string that includes:
        - "Omission": A complete sentence describing the missing detail.
        - "Severity": Classification level (Critical, Moderate, Optional).
        - "Reasoning": Explanation for the assigned severity.

    Parameters:
        omis (str): A string containing a list of omissions to be evaluated and classified.

    Returns:
        str: A CSV-formatted string containing the classification results for each omission.
    """
    
    deployment = 'gpt-4-32k'
    prompt = """
Please review the following list of omissions and determine whether each one represents information that should be included in the History of Present Illness (HPI) section of a medical note.

For each omission, classify it using the categories below based on the clinical significance and context:

Critical – Information that is essential for clinical decision-making or directly impacts patient safety.
Moderate – Information that adds important context or supports the patient-physician relationship, but does not critically affect immediate decision-making (e.g., patient concerns, preferences, or communication-related factors).
Optional – Additional details that may be helpful but are not necessary for understanding the patient’s condition or for making clinical decisions.

Output Format:
Your response must be a comma-delimited CSV. Use proper CSV escaping rules
 
The CSV must include the following columns:

"Omission" – A complete sentence describing the missing information. Enclose each entry in double quotes and follow standard CSV escaping rules.
"Severity" – The classification category (Critical, Moderate, or Optional). Enclose each entry in double quotes and follow standard CSV escaping rules.
"Reasoning" – A concise explanation of why the information was assigned its severity level. Enclose each entry in double quotes and follow standard CSV escaping rules.

Example of a well-formatted omission:
“The patient reported feeling faint after the prostate biopsy and experienced abdominal discomfort, which was relieved with Tylenol.”

Example of a poorly formatted omission (do not do this):
"Chest discomfort description clarified" – This lacks specific detail and is too vague.

    """
    messages=[
        {"role": "system", "content": 'You are a physician tasked with ensuring complete and accurate medical documentation.'},
        {"role": "user", "content": prompt + omis}
    ]
    response = client.chat.completions.create(
        model=deployment, 
        messages=messages,
    )
    return (response.choices[0].message.content)

In [206]:
def IdentifyOmission7(transc: str,
                      note: str):
    """
    Identifies clinically relevant information missing from the History of Present Illness (HPI) section 
    of a medical note by comparing it with the corresponding patient visit transcript.

    It returns a bullet-point list of omissions—details that are present in the transcript but missing from the 
    HPI—based. Only omissions that are clinically relevant, potentially impactful on 
    decision-making, or important to the patient-physician relationship are included.

    Parameters:
        transc (str): The transcript of the patient visit (dialogue between patient and provider).
        note (str): The HPI section of the medical note to compare against the transcript.

    Returns:
        str: A bullet-point list of missing clinically relevant details that should be added to the HPI.
    """
    deployment = 'gpt-4-32k'
    prompt1 = "Here is the note for a patient:"
    prompt2 = """ "Review the following patient visit transcript and identify any information that is present in the transcript but missing from the History of Present Illness (HPI). Do not add any additional information that is not in the transcript. Do not make assumptions. Only include details that should be in the HPI of a medical note. Present your findings as a simple bullet-point list.

Only include items that meet all of the following criteria:

Clinically relevant and necessary for a complete understanding of the patient's condition.

Potentially impactful on clinical decision-making or patient care.

Helpful in strengthening the patient-physician relationship.

Do not list omissions that do not meet these criteria."
"""
    
    omprompt2 = prompt1 + note + prompt2 + transc

    messages=[
        {"role": "system", "content": 'You are a physician who is writing a medical note'},
        {"role": "user", "content": omprompt2}
    ]
    response = client.chat.completions.create(
        model=deployment, 
        messages=messages,
    )
    return (response.choices[0].message.content)

For each row of the data file, will send the transcript to versa for reformatting.

In [53]:
def split_transcript_into_segments_df(transcript, segment_size=3000):
    """
    Split a transcript into segments of approximately the specified size,
    ensuring that words are not broken, and return as a DataFrame.
    
    Parameters:
    transcript (str): The input transcript text
    segment_size (int): Target size for each segment in characters
    
    Returns:
    pandas.DataFrame: DataFrame with segment_id and segment_text columns
    """
    segments = []
    
    # If transcript is shorter than segment_size, return it as is
    if len(transcript) <= segment_size:
        segments.append({"segment_id": 1, "segment_text": transcript})
        return pd.DataFrame(segments)
    
    # Start position for the current segment
    start = 0
    segment_id = 1
    
    while start < len(transcript):
        # Calculate end position (either segment_size or end of transcript)
        end = min(start + segment_size, len(transcript))
        
        # If we're not at the end of the transcript, find the last space to avoid breaking words
        if end < len(transcript):
            # Look for the last space within the segment
            while end > start and transcript[end] != ' ':
                end -= 1
            
            # If no space found (unlikely for normal text), just cut at segment_size
            if end == start:
                end = start + segment_size
        
        # Add the segment to our list with an ID
        segments.append({
            "segment_id": segment_id,
            "segment_text": transcript[start:end]
        })
        
        # Increment segment ID
        segment_id += 1
        
        # Move start position to the beginning of the next segment
        start = end
    
    # Convert list of dictionaries to DataFrame
    return pd.DataFrame(segments)

## Print out omissions with grade

In [211]:
## This saves the final dataframe to a CSV file
dir_save = r'YOUR_DIR'
file_save_name ='FinalOmissionsGraded.csv'
file_save_path = os.path.join(dir_save, file_save_name)

if os.path.exists(file_save_path):
    print(f"File {file_save_name} already exists. Skipping.")
else:
    final_try_df.to_csv(file_save_path, index=True)

In [213]:
## This saves each classification to its own file
dir_save = r'YOUR_DIR'
for index, row in final_try_df.iterrows():
    # Check if the 'Classified' column is not NA
    if pd.notna(final_try_df.at[index, 'Classified']):
        try:
            # Convert the string data to a DataFrame
            temp_df = pd.read_csv(io.StringIO(final_try_df.at[index, 'Classified']), header=0)
            
            # Create a unique filename for each row (using index)
            file_save_name = row['Encounter ID']  + '.csv'
        
            output_path = os.path.join(dir_save, file_save_name)
            
            # Save the DataFrame to CSV
            temp_df.to_csv(output_path, index=True)
            print(f"Saved data from row {index} to {file_save_name}")
            
        except Exception as e:
            print(f"Error processing row {index}: {e}")
    else:
        print(f"Row {index}: 'Classified' column is NA, skipping")

Saved data from row 0 to 058194cd-d4dc-490b-8ecf-9b68465e39a3.csv
Saved data from row 1 to 0a38cf2d-5c35-4f20-bb27-e29a87b6a526.csv
Saved data from row 2 to 12179291-2c2b-4f33-adae-4aecc037b9ce.csv
Saved data from row 3 to 1228c055-6838-473e-a581-3b8500af9817.csv
Saved data from row 4 to 2a4f5cf7-7142-4d03-a4dd-b1e2cf544599.csv
Saved data from row 5 to 41aa275e-3a4d-47e2-8239-6487be7bc059.csv
Saved data from row 6 to 43f704df-f272-406c-9137-98533a075ff8.csv
Saved data from row 7 to 5899f16f-17eb-45a3-a57b-ac276162de13.csv
Saved data from row 8 to 5b0e2f0d-8db8-4245-af07-7afe5564f4f1.csv
Saved data from row 9 to 7d32cae1-fab9-47d3-91a6-8e303acbdd0b.csv
Saved data from row 10 to 900696c1-6436-4241-8445-abeff09af04b.csv
Saved data from row 11 to 9164b014-e147-472d-8245-65ef72c67f00.csv
Saved data from row 12 to b0ccc115-b9c3-4b57-a010-60976fd53789.csv
Saved data from row 13 to c29c8073-b3ff-418a-9be1-92d323540e4c.csv
Saved data from row 14 to d7d6dab6-d22b-434e-9c30-da35a64882bc.csv


In [19]:
# Reload the results from backup

dir_save = r'YOUR_DIR'
file_save_name ='FinalOmissionsGraded.csv'
file_save_path = os.path.join(dir_save, file_save_name)
final_try_df = pd.read_csv(file_save_path)

#back_up_file = r"BACKUP_OMISSION_FILE.csv"
#final_try_df = pd.read_csv(back_up_file)

In [26]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping
pd.set_option('display.max_colwidth', None)  # Show full content in each cell

for index, row in final_try_df.iterrows():
    print(index)
    if pd.notna(final_try_df.at[index, 'Classified']):
        # Parse the CSV string into a dataframe
        temp_df = pd.read_csv(io.StringIO(final_try_df.at[index, 'Classified']), header=0)
        
        # Apply redactions to all string columns in temp_df
        for col in temp_df.columns:
            if temp_df[col].dtype == 'object':  # Only process text columns
                temp_df[col] = temp_df[col].apply(lambda text: redact_phi(str(text)) if pd.notna(text) else text)
        
        display(temp_df)

0


,,```csv
Omission,Severity,Reasoning
"Patient referenced experiencing sexual abuse in childhood, particularly by his wife's stepfather, and provided details about how his family and he coped with it. He mentioned scrubbing himself in the bath as a reaction to the abuse.",Moderate,"This historical trauma significantly adds context to the patient's psychological state and coping mechanisms, but it may not critically affect immediate decision-making unless linked to current symptoms."
Patient discussed the family's handling of trauma and its potential influence on his psychological state and irritability.,Moderate,Important for contextualizing the patient's psychological state and identifying contributing factors to irritability; supports a biopsychosocial approach without being critical for acute decision-making.
"Patient provided insights regarding a significant recent stressor: his son’s medical emergency involving a burst polyp. He described the high stress of the event, including his son's autism and communication difficulties, challenges navigating the emergency, and lack of sleep, all of which may have contributed to his irritability.",Critical,"This recent stressor directly impacts the patient’s current mental and physical state, including irritability and potentially sleep deprivation, which are critical to assess for immediate management."
Interest in how anxiety may contribute to irritability and speculation that his irritability could stem from anxiety or underlying brain/chemical makeup.,Optional,This reflects patient speculation and curiosity rather than a definitive element that requires immediate mitigation or decision-making.
Stated preference for current counseling sessions to be frequent given his current situation; also noted past discussions about counseling referral and the familiarity with related clinicians in the area.,Moderate,This preference and familiarity with clinicians influence therapeutic planning and the patient-physician relationship but does not critically alter immediate care.
"Context of his low heart rate as typical for him due to a 'runner’s heart,' contrasted with recent elevated heart rate during stressful periods (e.g., high-stress scenarios like his son's emergency). He also noted his recent morning blood pressure readings were normal, such as 117/68 and 114/65.",Critical,"Current vital signs and patterns are essential for assessing physical response to stress, ruling out underlying conditions, and ensuring patient safety in light of elevated heart rate during stressful events."
"Expressed appreciation for work and community feedback: His job outdoors contributes positively to his mood, with frequent community acknowledgment providing a positive sense of contribution and satisfaction.",Optional,This provides helpful psychosocial context for understanding the patient's life satisfaction but does not impact immediate clinical decision-making.
"Specific details about initiating doubling the mirtazapine dose (15mg), patient agreement, and possible side effects discussed (e.g., increased fatigue and appetite stimulation). Instructions for transitioning the medication were given.",Critical,"Medication adjustments and discussions about side effects directly impact clinical care and patient safety, necessitating inclusion for informed decision-making and follow-up."
Patient denied any suicidal thoughts but emphasized enjoying his job and pickleball as important pleasant activities in his life.,Critical,"Denial of suicidal thoughts is crucial for assessing patient safety, and identifying protective factors like work and hobbies helps guide mental health care planning."


1


,,```csv
Omission,Severity,Reasoning
"Patient reports ongoing shortness of breath when walking distances, described as 'huffing and puffing;' denies worsening shortness of breath over the past 10 days or new symptoms when lying flat.",Critical,"Shortness of breath is a common and potentially serious symptom that requires evaluation, particularly if associated with heart or lung pathology. Context about worsening symptoms or positional changes directly impacts clinical decision-making."
Patient states recent weight gain may also correlate with bilateral lower extremity swelling.,Moderate,"Weight gain and swelling are important in understanding possible fluid retention or heart failure. While not immediately dangerous in isolation, the information adds important context to guide treatment planning."
Swelling has progressed to include pain with a 'nerve-like' pins and needles sensation primarily in thighs and calves. Pain is most noticeable during ambulation but does not inhibit walking.,Moderate,The description of progressive pain and swelling with sensory changes provides valuable diagnostic clues but does not immediately impact urgent clinical decisions unless linked to acute pathology.
"Reports urinary and fecal incontinence, with improvement through physical therapy sessions.",Moderate,"This information reflects progress and provides a baseline for functional status, which is important for patient care but less critical for immediate decision-making unless there is acute worsening."
Notes episodes of abdominal tenderness without palpable mass or other findings during exam.,Optional,Abdominal tenderness without other findings is unlikely to change immediate management unless paired with more concerning symptoms or exam findings.
Patient previously trialed carvedilol approximately 1 week ago but experienced immediate severe diarrhea and discontinued usage.,Critical,"Adverse effects from medications like carvedilol directly affect possible treatments and guidelines for future prescribing, making this essential for patient safety."
Expresses uncertainty about the recent prescription for lisinopril from another physician despite prior discontinuation due to hyperkalemia.,Critical,"Uncertainty about prescribed medications and prior contraindications (e.g., hyperkalemia with lisinopril) directly impacts patient safety and treatment planning, requiring meticulous documentation."
Patient confirmed no changes in medications within the last two weeks; continues diltiazem at home and recognizes its absence from his active medications list in recent notes.,Moderate,"The absence of diltiazem from the active medication list could lead to errors and should be reconciled, though this is more relevant for documentation accuracy than immediate clinical decisions."
"Reports difficulty distinguishing sounds during communication, consistent with prior assessment of possible hearing loss, but has not pursued hearing aids despite prior testing indicating they could be helpful.",Optional,Hearing loss is relevant for the patient’s quality of life and communication but does not critically affect clinical decision-making unless it impairs communication of symptoms.


2


...

,,```csv
Omission,Severity,Reasoning
The patient's exact blood pressure readings on admission to the ED are missing from the HPI.,Critical,"Specific blood pressure values are essential for assessing the severity of hypertension and guiding immediate management decisions, especially in the context of chest discomfort."
"The patient's description of chest discomfort, perceived as agitation related to high blood pressure, is absent.",Moderate,"This distinction provides important context on the nature of the chief complaint and prevents misinterpretation of symptoms, which is important for patient-centered care but does not critically alter immediate management."
The integration of relevant EKG findings such as sinus tachycardia (HR 103) and T-wave inversions in lead 3 into the HPI is missing.,Critical,"EKG findings directly influence the diagnostic approach to chest discomfort and elevated blood pressure, particularly in ruling out or identifying acute cardiac conditions."
```,NaN,NaN


13


,,```csv
Omission,Severity,Reasoning
"The pain is localized in the right buttock and worsens with certain movements (e.g., pushing outward with the knee or specific positions during examination).",Critical,"Localized pain and positional triggers are essential to assess for conditions like sciatica, musculoskeletal injury, or hip pathology, which impact treatment decisions."
The pain worsened significantly after the fall but has since slightly improved.,Critical,"A history of trauma is crucial to evaluate for fractures, soft tissue injuries, or progression of a chronic condition."
"Difficulty bearing weight on the affected leg, noting that her hip pain remains pronounced despite improvement.",Critical,Weight-bearing difficulty and persistent pain are critical to assess for potential fractures or injuries affecting mobility and safety.
"Previously noted compression fractures in her spine, which may be associated with her chronic hip pain.",Moderate,"Compression fractures provide context for chronic pain, aiding in differential diagnosis and highlighting potential musculoskeletal contributions."
Pressure sensation and itching at the lesion site without significant bone pain.,Moderate,"These symptoms provide context for evaluating infection, dermatologic, or soft tissue conditions that may impact management strategies."
No notable changes in the size or behavior of the lesion.,Optional,Stability of the lesion is helpful information but does not critically change clinical management based on the current presentation.
"Persistent drainage since February, with yellow thick pus occasionally exacerbated by episodes of pressure buildup.",Critical,Persistent drainage and purulent discharge are key indicators of infection that require intervention to prevent complications.
Patient mentioned using hydrocortisone cream for itching relief.,Optional,Topical hydrocortisone use provides minor context for self-management but is not essential for understanding the clinical picture.
Burning sensation worsens when eating and swallowing but is present at baseline.,Moderate,"This symptom is important for evaluating dyspepsia or GERD, which may impact quality of life but is unlikely to require emergency treatment."


14


,,```csv
Omission,Severity,Reasoning
"The patient's history of sensation of tissue pushing out of the vagina when standing for extended periods, described as 'something wants to come out,' which could be clinically relevant to pelvic issues.",Critical,"This information is essential for clinical decision-making as it strongly suggests pelvic organ prolapse, a condition that requires further evaluation and may have significant implications for treatment."
"The patient reports a history of occasional urinary symptoms or bother, though this was inconsistent.",Moderate,"This information could add important context by supporting a possible association with urological or pelvic conditions, but its inconsistency makes it less critical for immediate clinical decision-making."
Family history details indicating that the patient's daughter is unclear about additional potential kidney abnormalities or bleeding sources.,Optional,This information could be helpful for understanding genetic or familial predispositions but does not directly affect the patient's current clinical condition or immediate care needs.
```,NaN,NaN
